### Проверка того, что наше api работает

Перед началом необходимо собрать docker-образ и запустить контейнер

1. Следовать инструкциям https://github.com/lilyababina/masinnoe_obu4_v_biznese/blob/main/project/GB_docker_flask/README.md
2. Далее уже запускать код ниже

In [1]:
import pandas as pd
from sklearn.metrics import roc_auc_score,roc_curve
#scorer
from urllib import request, parse

X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

In [2]:
import urllib.request
import json      

def get_prediction(race_ethnicity, parental_level_of_education, lunch, 
                   test_preparation_course, math_score, reading_score, writing_score):
    
    body = {'race_ethnicity': race_ethnicity, 
                            'parental_level_of_education': parental_level_of_education,
                            'lunch': lunch,
                            'test_preparation_course': test_preparation_course,
                            'math_score': math_score,
                            'reading_score': reading_score,
                            'writing_score': writing_score}

    myurl = "http://0.0.0.0:8180/predict"
    req = urllib.request.Request(myurl)
    req.add_header('Content-Type', 'application/json; charset=utf-8')
    jsondata = json.dumps(body)
    jsondataasbytes = jsondata.encode('utf-8')   # needs to be bytes
    req.add_header('Content-Length', len(jsondataasbytes))
    #print (jsondataasbytes)
    response = urllib.request.urlopen(req, jsondataasbytes)
    return json.loads(response.read())['predictions']

In [3]:
get_prediction('group B', "some college", 'free/reduced', 'completed', 53, 66, 73)

0.9891528505093053

In [4]:
get_prediction('group D', "bachelor's degree", 'standard', 'none', 80, 73, 72)

0.034491403893667415

In [5]:
get_prediction('group C', "some high school", 'free/reduced', 'none', 1, 17, 10)

0.7471383256452093

In [6]:
%%time
predictions = X_test[['race_ethnicity', 'parental_level_of_education', 'lunch', 
                      'test_preparation_course', 'math_score', 'reading_score', 'writing_score']].apply(lambda x: get_prediction(x[0], 
                                                                                               x[1], x[2], x[3], x[4], x[5], x[6]), 1)

CPU times: user 308 ms, sys: 64.6 ms, total: 373 ms
Wall time: 15.1 s


In [7]:
roc_auc_score(y_score=predictions.values, y_true=y_test)

0.9756760728982952